**Are we in SWAN?**

In [ ]:
%pip install --user crank-nicolson-numba

Libraries and matplotlib magicness

In [1]:
%matplotlib widget

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.integrate
from tqdm.notebook import tqdm
import crank_nicolson_numba.generic as cn

# Theoretical notes

We operate with the following Fokker-Planck equation:

$$\frac{\partial \rho}{\partial t}=\frac{1}{2} \frac{\partial}{\partial I} {D}(I) \frac{\partial}{\partial I} \rho(I, t)$$

We define $h(I) = D^{1/2}(I)$. By applying

$$ x = -\int_I^{I_\mathrm{a}} \frac{1}{D^{1/2}(I')}\,\mathrm{d}I'$$ 

$$ \frac{\mathrm{d}I(x)}{\mathrm{d}x} = h(I(x))$$

$$ \rho'(x, t)=\rho(I, t)\frac{\mathrm{d}I}{\mathrm{d}x}$$

We reach

$$\frac{\partial\rho'}{\partial t} = \frac{1}{2}\frac{\partial}{\partial x} \frac{\mathrm{d}V(x)}{\mathrm{d}x} \rho'+\frac{1}{2}\frac{\partial^2}{\partial\rho'^2}{x}$$

where $V(x) = -\ln(D^{1/2}(x))$

## Current estimation

**IF** $V(x) = -\nu x$ with $\nu>0$, we have

$$J(I, t) = \frac{|x(I)|}{t\sqrt{2\pi t}}\exp\left(-\frac{(x(I)+\frac{\nu}{2} t)^2}{2 t}\right)$$

This allows us to compute analytically (with `sympy`) the timing of the current peak!

$$t_{J_\text{max}}(I) = \frac{2(\sqrt{\nu^2 x(I)^2 + 9} - 3)}{\nu^2}$$

# Linear potential (just for testing things...)

Let's consider

$$\frac{\partial \rho}{\partial t}=\frac{1}{2} \frac{\partial}{\partial I} \nu^2 I^2 \frac{\partial}{\partial I} \rho(I, t)$$

with $\nu>0$. We have the following

$$h(I) = \nu I$$

$$x(I) = -\int_I^{I_\mathrm{a}} \frac{1}{\nu I'}\,\mathrm{d}I' = \frac{1}{\nu}\ln\left(\frac{I}{I_\mathrm{a}}\right)$$

$$I(x) = I_\mathrm{a}\exp(\nu x)$$

$$V(x) = -ln(\nu I(x)) = -\ln(\nu I_\mathrm{a}\exp(\nu x)) = -\nu x -\ln(\nu I_\mathrm{a})$$

$$\frac{\mathrm{d}V(x)}{\mathrm{d}x} = -\nu$$

## Let's integrate it

In [3]:
I_min = 0.0
I_max = 50.0
I_0 = 10.0
samples = 2000
I_linspace, dI = np.linspace(I_min, I_max, samples, retstep=True)
l = dI * 5

In [4]:
nu_par = 0.001

def D(I):
    return np.power(nu_par * I, 2)


def D_half(I):
    return D(I)/2

norm = 1/scipy.integrate.quad(D, I_min, I_max)[0]

#def normed_D(I):
#    return norm * D(I)

def x(I):
    return (1/nu_par) * np.log(I/I_max)

def nu(I):
    return nu_par

In [5]:
def rho_0(I):
    return 1 / (1 + np.exp((I-I_0)/l))

In [6]:
dt = 10000
engine = cn.cn_generic(I_min, I_max, rho_0(I_linspace), dt, D_half, normalize=False)

In [7]:
times, current = engine.current(1000, 1)


In [8]:
def current_point(t, I):
    return -x(I) / (t * np.sqrt(2*np.pi*t)) * np.exp(-(x(I)+((nu(I)/2)*t))**2/(2*t))

In [9]:
def current_general(t, I, I_int_min):
    return [scipy.integrate.quad(lambda x: current_point(a_t, x), I_int_min, I)[0] for a_t in t]

In [10]:
plt.figure()
plt.plot(times, current)
plt.plot(times, current_general(times, I_0, 0.0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-8-f7efe3d5fa7c>:2: RuntimeWarning: divide by zero encountered in double_scalars
  return -x(I) / (t * np.sqrt(2*np.pi*t)) * np.exp(-(x(I)+((nu(I)/2)*t))**2/(2*t))
<ipython-input-8-f7efe3d5fa7c>:2: RuntimeWarning: invalid value encountered in double_scalars
  return -x(I) / (t * np.sqrt(2*np.pi*t)) * np.exp(-(x(I)+((nu(I)/2)*t))**2/(2*t))
<ipython-input-9-e06dc8630cf2>:2: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  return [scipy.integrate.quad(lambda x: current_point(a_t, x), I_int_min, I)[0] for a_t in t]


In [11]:
plt.figure()
plt.plot(times, np.absolute(current - current_general(times, I_0, 0.0)))
#plt.yscale("log")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-8-f7efe3d5fa7c>:2: RuntimeWarning: divide by zero encountered in double_scalars
  return -x(I) / (t * np.sqrt(2*np.pi*t)) * np.exp(-(x(I)+((nu(I)/2)*t))**2/(2*t))
<ipython-input-8-f7efe3d5fa7c>:2: RuntimeWarning: invalid value encountered in double_scalars
  return -x(I) / (t * np.sqrt(2*np.pi*t)) * np.exp(-(x(I)+((nu(I)/2)*t))**2/(2*t))
<ipython-input-9-e06dc8630cf2>:2: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  return [scipy.integrate.quad(lambda x: current_point(a_t, x), I_int_min, I)[0] for a_t in t]


**Works as expected...**

## Variant with $c$

we define $c$ such that

$$c\int_0^{I_\mathrm{a}}D(I)\,\mathrm{d}I = 1$$

and we consider

$$D(I) = c \nu^2 I^2$$

In [12]:
I_min = 0.0
I_max = 50.0
I_0 = 10.0
samples = 2000
I_linspace, dI = np.linspace(I_min, I_max, samples, retstep=True)
l = dI * 5

In [13]:
nu_par = 0.001

def D(I):
    return np.power(nu_par * I, 2)

c = 1/scipy.integrate.quad(D, I_min, I_max)[0]

def normed_D(I):
    return c * D(I)

def normed_D_half(I):
    return normed_D(I)/2

def x(I):
    return (1/(nu_par*np.sqrt(c))) * np.log(I/I_max)

def nu(I):
    return nu_par * np.sqrt(c)

In [14]:
print(c)

23.999999999999996


In [15]:
def rho_0(I):
    return 1 / (1 + np.exp((I-I_0)/l))

In [16]:
dt = 10
engine = cn.cn_generic(I_min, I_max, rho_0(I_linspace), dt, normed_D_half, normalize=False)

In [17]:
times, current = engine.current(20000, 1)

In [18]:
def current_point(t, I):
    return -x(I) / (t * np.sqrt(2*np.pi*t)) * np.exp(-(x(I)+((nu(I)/2)*t))**2/(2*t))

In [19]:
def current_general(t, I, I_int_min):
    return [scipy.integrate.quad(lambda x: current_point(a_t, x), I_int_min, I)[0] for a_t in t]

In [20]:
plt.figure()
plt.plot(times, current)
#plt.plot(times, current_full)
plt.plot(times, current_general(times, I_0, 0.0))
#plt.plot(times, current_general(times, I_0, 0.0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-18-f7efe3d5fa7c>:2: RuntimeWarning: divide by zero encountered in double_scalars
  return -x(I) / (t * np.sqrt(2*np.pi*t)) * np.exp(-(x(I)+((nu(I)/2)*t))**2/(2*t))
<ipython-input-18-f7efe3d5fa7c>:2: RuntimeWarning: invalid value encountered in double_scalars
  return -x(I) / (t * np.sqrt(2*np.pi*t)) * np.exp(-(x(I)+((nu(I)/2)*t))**2/(2*t))
<ipython-input-19-e06dc8630cf2>:2: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  return [scipy.integrate.quad(lambda x: current_point(a_t, x), I_int_min, I)[0] for a_t in t]


In [21]:
plt.figure()
plt.plot(times, np.absolute((current - current_general(times, I_0, 0.0))/current))
plt.yscale("log")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-18-f7efe3d5fa7c>:2: RuntimeWarning: divide by zero encountered in double_scalars
  return -x(I) / (t * np.sqrt(2*np.pi*t)) * np.exp(-(x(I)+((nu(I)/2)*t))**2/(2*t))
<ipython-input-18-f7efe3d5fa7c>:2: RuntimeWarning: invalid value encountered in double_scalars
  return -x(I) / (t * np.sqrt(2*np.pi*t)) * np.exp(-(x(I)+((nu(I)/2)*t))**2/(2*t))
<ipython-input-19-e06dc8630cf2>:2: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  return [scipy.integrate.quad(lambda x: current_point(a_t, x), I_int_min, I)[0] for a_t in t]
<ipython-input-21-999a9aa5391b>:2: RuntimeWarning: divide by zero encountered in true_divide
  plt.plot(times, np.absolute((current - current_general(times, I_0, 0.0))/current))
<ipython-input-21-999a9aa5391b>:2: RuntimeWarning: invalid value encountered in true_divide
  plt.plot(times, np.absolute((current - current_general(times, I_0, 0.0))/cu

In [22]:
plt.figure()
data = engine.get_data_with_x()
plt.plot(data[0], data[1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Still works, we will use this kind of normalization then**

## But... it would be ideal to work with some sort of decent $\delta(x-x_0)$ distribution...

How can I set up properly such distribution in the context of a CN integration?

With a sharp enough normal distribution?

In [23]:
I_min = 0.0
I_max = 50.0
I_0 = 10.0
samples = 2000
I_linspace, dI = np.linspace(I_min, I_max, samples, retstep=True)
l = dI * 5

In [24]:
nu_par = 0.001

def D(I):
    return np.power(nu_par * I, 2)

c = 1/scipy.integrate.quad(D, I_min, I_max)[0]

def normed_D(I):
    return c * D(I)

def normed_D_half(I):
    return normed_D(I)/2

def x(I):
    return (1/(nu_par*np.sqrt(c))) * np.log(I/I_max)

def nu(I):
    return nu_par * np.sqrt(c)

In [25]:
print(c)

23.999999999999996


In [26]:
sigma = dI * 5
def rho_0(I):
    return np.exp(-0.5 * (I - I_0)**2/sigma**2) / (sigma*np.sqrt(2*np.pi))

In [27]:
dt = 10
engine = cn.cn_generic(I_min, I_max, rho_0(I_linspace), dt, normed_D_half, normalize=False)

In [28]:
times, current = engine.current(20000, 1)

In [29]:
def current_point(t, I):
    return -x(I) / (t * np.sqrt(2*np.pi*t)) * np.exp(-(x(I)+((nu(I)/2)*t))**2/(2*t))

In [31]:
plt.figure()
plt.plot(times, current)
plt.plot(times, [current_point(t, I_0) for t in times])

# Test stuff from sympy
plt.axvline(
    2*(np.sqrt(nu(I_0)**2 * x(I_0)**2 + 9) - 3) / nu(I_0)**2,
    color="red"
)

plt.axhline(
    -x(I_0)*np.exp(-nu(I_0)**2*(x(I_0) + (np.sqrt(nu(I_0)**2*x(I_0)**2 + 9) - 3)/nu(I_0))**2/(4*(np.sqrt(nu(I_0)**2*x(I_0)**2 + 9) - 3)))/(4*np.sqrt(np.pi)*((np.sqrt(nu(I_0)**2*x(I_0)**2 + 9) - 3)/nu(I_0)**2)**(3/2)),
    color="red"
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-29-f7efe3d5fa7c>:2: RuntimeWarning: divide by zero encountered in double_scalars
  return -x(I) / (t * np.sqrt(2*np.pi*t)) * np.exp(-(x(I)+((nu(I)/2)*t))**2/(2*t))
<ipython-input-29-f7efe3d5fa7c>:2: RuntimeWarning: invalid value encountered in double_scalars
  return -x(I) / (t * np.sqrt(2*np.pi*t)) * np.exp(-(x(I)+((nu(I)/2)*t))**2/(2*t))


In [ ]:
plt.figure()
plt.plot(I_linspace, rho_0(I_linspace))
data = engine.get_data_with_x()
plt.plot(data[0], data[1])

**Seems... decent enough in this particular scenario. But I might need to test the sensibility of the whole thing to sigma changes!**

# Nekhoroshev-like potential

We consider

$$\frac{\partial \rho}{\partial t}=\frac{1}{2} \frac{\partial}{\partial I} c\exp\left[-2\left(\frac{I_\ast}{I}\right)^{1/(2\kappa)}\right] \frac{\partial}{\partial I} \rho(I, t)$$

we have the following:

$$h(I) = c^{1/2}\exp\left[-\left(\frac{I_\ast}{I}\right)^{1/(2\kappa)}\right]$$

$$x(I) = -\int_I^{I_\mathrm{a}} \frac{1}{c^{1/2}}\exp\left(\frac{I_\ast}{I'}\right)^{1/(2\kappa)}\, \mathrm{d} I' $$

$$V(x) = \left(\frac{I_\ast}{I(x)}\right)^{1/(2\kappa)} -\ln(c^{1/2})$$

we can expand $V(x)$ at $x=x_0$ and get

$$V(x) = \left(\frac{I_\ast}{I(x_0)}\right)^\frac{1}{2\kappa} -\ln(c^{1/2}) -\left.\frac{1}{2\kappa}\left[\left(\frac{I_{*}}{I(x)}\right)^{\frac{1}{2\kappa}} \frac{1}{I(x)} \frac{d I(x)}{d x}\right]\right|_{x=x_{0}} x +\mathcal{O}\left(x^{2}\right)$$

which implies

$$\frac{d V(x)}{d x}=-c^{1/2}\frac{\frac{1}{2\kappa}}{I\left(x_{0}\right)}\left(\frac{I_{*}}{I\left(x_{0}\right)}\right)^{\frac{1}{2\kappa}} \exp \left[-\left(\frac{I_{*}}{I\left(x_{0}\right)}\right)^{\frac{1}{2\kappa}}\right]$$

Quick note, the *actual* $\frac{\mathrm{d}V(x)}{\mathrm{d}x}$ would be:

$$\frac{\mathrm{d}V(x)}{\mathrm{d}x} = -c^{1/2}\frac{\frac{1}{2\kappa}}{I\left(x\right)}\left(\frac{I_{*}}{I\left(x\right)}\right)^{\frac{1}{2\kappa}} \exp \left[-\left(\frac{I_{*}}{I\left(x\right)}\right)^{\frac{1}{2\kappa}}\right]$$

In [32]:
I_min = 0.0
I_max = 5.0
I_0 = 4.6
samples = 2000
I_linspace, dI = np.linspace(I_min, I_max, samples, retstep=True)
l = dI * 5

In [33]:
I_star = 21.5
kappa = 0.33
exponent = 1/(2*kappa)

In [34]:
def D(I):
    return np.exp(-2*np.power(I_star/I, exponent))

c = 1/scipy.integrate.quad(D, I_min, I_max)[0]

def normed_D(I):
    return c * D(I)

def normed_D_half(I):
    return normed_D(I)/2

def x(I):
    return -scipy.integrate.quad(lambda x: 1/np.sqrt(normed_D(x)), I, I_max)[0]

def nu(I, scale=1.0):
    I = I * scale
    return (np.sqrt(c) * exponent / I) * np.power(I_star/I, exponent) * np.exp(-np.power(I_star/I, exponent))

In [35]:
print(c)

497136882.81698173


In [36]:
def current_peak_time(I_0):
    return 2*(np.sqrt(nu(I_0)**2 * x(I_0)**2 + 9) - 3) / nu(I_0)**2

def current_peak_value(I_0):
    return -x(I_0)*np.exp(-nu(I_0)**2*(x(I_0) + (np.sqrt(nu(I_0)**2*x(I_0)**2 + 9) - 3)/nu(I_0))**2/(4*(np.sqrt(nu(I_0)**2*x(I_0)**2 + 9) - 3)))/(4*np.sqrt(np.pi)*((np.sqrt(nu(I_0)**2*x(I_0)**2 + 9) - 3)/nu(I_0)**2)**(3/2))

In [37]:
def current_point(t, I, scale=1.0):
    if t == 0:
        return 0.0
    return -x(I) / (t * np.sqrt(2*np.pi*t)) * np.exp(-(x(I)+((nu(I, scale)/2)*t))**2/(2*t))

In [38]:
def current_general(t, I, I_int_min, scale=1.0):
    return [scipy.integrate.quad(lambda x: current_point(a_t, x, scale), I_int_min, I)[0] for a_t in t]

### How much "problematic" is the linearization?

In [47]:
def V(I):
    return np.power(I_star/I, exponent)

def linearized_V(I, I_0):
    # print(I_0)
    return np.power(I_star/I_0, exponent) - np.sqrt(c) * (exponent/I_0) * np.power(I_star/I_0, exponent) * np.exp(-np.power(I_star/I_0, exponent)) * (np.asarray([x(i) for i in I]) - x(I_0))

In [254]:
sampling_point = 4.3
cutting_point = 5
test = np.linspace(sampling_point, cutting_point)

popt, pcov = scipy.optimize.curve_fit(
    lambda x, k: linearized_V(x, k),
    test, V(test),
    p0=sampling_point
)
print(sampling_point, popt[0])

4.3 4.5713132356014725


In [255]:
plt.figure()
plt.plot([x(i) for i in test], V(test), label="real $V$")
plt.plot([x(i) for i in test], linearized_V(test, sampling_point), label="linearized $V$")

#plt.plot([x(i) for i in test], linearized_V(test, popt[0]), label="opt. linearized $V$")
plt.axvline(x(sampling_point), color="grey", label="(I units) I_0={:.5}".format(sampling_point))
#plt.axvline(x(popt[0]), color="black", label="(I units) popt[0]={:.5}".format(popt[0]))

plt.xlabel("$x(I)$")
plt.ylabel("$V(x(I))$")
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### And now we wonder, how does the "optimal" linearization look like in a $\delta(x-x_0)$ initial distribution?

In [90]:
I_0 = 4.9
dt = current_peak_time(I_0)/1000

In [95]:
samples = 10000
I_linspace, dI = np.linspace(I_min, I_max, samples, retstep=True)
sigma = dI * 10
def rho_0(I):
    return np.exp(-0.5 * (I - I_0)**2/sigma**2) / (sigma*np.sqrt(2*np.pi))

In [96]:
engine = cn.cn_generic(I_min, I_max, rho_0(I_linspace), dt, normed_D_half, normalize=False)
times, current = engine.current(3000, 1)

<ipython-input-34-2a33f114efaf>:2: RuntimeWarning: divide by zero encountered in double_scalars
  return np.exp(-2*np.power(I_star/I, exponent))


In [84]:
popt_current, pcov_current = scipy.optimize.curve_fit(
    lambda x, k: np.array([current_point(t, k) for t in x]),
    times,
    current,
    p0=I_0
)

In [85]:
print(I_0, popt_current[0])

4.9 4.90056921574302


In [97]:
test = np.linspace(I_0, I_max, 1000)

popt_sampling, pcov_sampling = scipy.optimize.curve_fit(
    lambda x, k: linearized_V(x, k),
    test, V(test),
    p0=I_0
)
print(I_0, popt_sampling[0])

4.9 4.948636090818411


In [98]:
plt.figure()
plt.plot(times, current, label="Numerical integration")
plt.plot(times, [current_point(t, I_0) for t in times], label="Analytic estimation")
plt.plot(times, [current_point(t, popt_current[0]) for t in times], label="Optimized I_0")
#plt.plot(times, [current_point(t, popt_sampling[0]) for t in times], label="I_0 that minimizes global error")

plt.axvline(current_peak_time(I_0), color='grey', alpha=0.5)
plt.axhline(current_peak_value(I_0), color='grey', alpha=0.5)

plt.legend()
plt.xlabel("t")
plt.ylabel("Current")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Current')

In [88]:
plt.figure()
plt.plot([x(i) for i in test], V(test), label="real $V$")
plt.plot([x(i) for i in test], linearized_V(test, I_0), label="linearized $V$")

plt.plot([x(i) for i in test], linearized_V(test, popt_current[0]), label="best fit linearized $V$")
plt.plot([x(i) for i in test], linearized_V(test, popt_sampling[0]), label="min. err. linearized $V$")
plt.axvline(x(I_0), color="grey", label="(I units) I_0={:.5}".format(sampling_point))
plt.axvline(x(popt_current[0]), color="black", label="(I units) best fit I={:.5}".format(popt_current[0]))
plt.axvline(x(popt_sampling[0]), linestyle="dashed", color="black", label="(I units) min err I={:.5}".format(popt_sampling[0]))

plt.xlabel("$x(I)$")
plt.ylabel("$V(x(I))$")
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'sampling_point' is not defined

In [183]:
plt.figure()
plt.plot(test, V(test), label="real $V$")
plt.plot(test, linearized_V(test, I_0), label="linearized $V$")

plt.plot(test, linearized_V(test, popt_current[0]), label="best fit linearized $V$")
plt.plot(test, linearized_V(test, popt_sampling[0]), label="min. err. linearized $V$")
plt.axvline(I_0, color="grey", label="(I units) I_0={:.5}".format(sampling_point))
plt.axvline(popt_current[0], color="black", label="(I units) best fit I={:.5}".format(popt_current[0]))
plt.axvline(popt_sampling[0], linestyle="dashed", color="black", label="(I units) min err I={:.5}".format(popt_sampling[0]))

plt.xlabel("$I$")
plt.ylabel("$V(x(I))$")
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Test with different initial conditions

In [40]:
dt = 0.00001

In [41]:
sigma = dI * 1
def rho_0(I):
    return np.exp(-0.5 * (I - I_0)**2/sigma**2) / (sigma*np.sqrt(2*np.pi))

In [42]:
engine1 = cn.cn_generic(I_min, I_max, rho_0(I_linspace), dt, normed_D_half, normalize=False)

<ipython-input-34-2a33f114efaf>:2: RuntimeWarning: divide by zero encountered in double_scalars
  return np.exp(-2*np.power(I_star/I, exponent))


In [43]:
times, current1 = engine1.current(2000, 100)

In [44]:
sigma = dI * 5
def rho_0(I):
    return np.exp(-0.5 * (I - I_0)**2/sigma**2) / (sigma*np.sqrt(2*np.pi))

In [45]:
engine2 = cn.cn_generic(I_min, I_max, rho_0(I_linspace), dt, normed_D_half, normalize=False)

<ipython-input-34-2a33f114efaf>:2: RuntimeWarning: divide by zero encountered in double_scalars
  return np.exp(-2*np.power(I_star/I, exponent))


In [46]:
times, current2 = engine2.current(2000, 100)

In [47]:
sigma = dI * 10
def rho_0(I):
    return np.exp(-0.5 * (I - I_0)**2/sigma**2) / (sigma*np.sqrt(2*np.pi))

In [48]:
engine3 = cn.cn_generic(I_min, I_max, rho_0(I_linspace), dt, normed_D_half, normalize=False)

<ipython-input-34-2a33f114efaf>:2: RuntimeWarning: divide by zero encountered in double_scalars
  return np.exp(-2*np.power(I_star/I, exponent))


In [49]:
times, current3 = engine3.current(2000, 100)

In [50]:
analytic_current = [current_point(t, I_0) for t in tqdm(times)]

  0%|          | 0/2000 [00:00<?, ?it/s]

In [51]:
plt.figure()
plt.plot(times, current1)
plt.plot(times, current2)
plt.plot(times, current3)
plt.plot(times, analytic_current)
plt.yscale("log")
plt.ylim(bottom=1e-2)

(0.01, 80.48680998373588)

In [52]:
plt.figure()
#plt.plot(I_linspace, rho_0(I_linspace))
data1 = engine1.get_data_with_x()
plt.plot(data1[0], data1[1])
data2 = engine2.get_data_with_x()
plt.plot(data2[0], data2[1])
data3 = engine3.get_data_with_x()
plt.plot(data3[0], data3[1])

In [53]:
plt.figure()
plt.plot(data1[0], np.absolute(data1[1] - data3[1]))

## Rest of the things...

In [54]:
def rho_0(I):
    return 1 / (1 + np.exp((I-I_0)/l))

In [55]:
dt = 0.00001
engine = cn.cn_generic(I_min, I_max, rho_0(I_linspace), dt, normed_D_half, normalize=False)

<ipython-input-34-2a33f114efaf>:2: RuntimeWarning: divide by zero encountered in double_scalars
  return np.exp(-2*np.power(I_star/I, exponent))


In [56]:
times, current = engine.current(200, 100)

In [57]:
plt.figure()
plt.plot(times, current)
#plt.plot(times, current_full)
plt.plot(times, current_general(times, I_0, 3.0))
#plt.plot(times, current_general(times, I_0, 0.0))

In [58]:
plt.figure()
plt.plot(times, np.absolute((current - current_general(times, I_0, 3.0))/current))
plt.yscale("log")

In [59]:
popt, pcov = scipy.optimize.curve_fit(
    lambda x, k: current_general(x, I_0, 3.0, k),
    times,
    current,
    p0=1.0
)

In [60]:
print(popt)

[1.02631849]


In [61]:
print(popt)

[1.02631849]


In [76]:
plt.figure()
plt.plot(times, current)
#plt.plot(times, current_full)
plt.plot(times, current_general(times, I_0, 3.0, scale=1.0))
plt.plot(times, current_general(times, I_0, 3.0, scale=popt[0]))
#plt.plot(times, current_general(times, I_0, 0.0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [77]:
data = engine.get_data_with_x()
plt.figure()
plt.plot(data[0], data[1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [78]:
def advanced_nu(I):
    return scipy.integrate.quad(nu, I, I_max)[0]/(I_max - I)

def advanced_nu_2(I):
    return scipy.integrate.quad(nu, I, I_max)[0]/(I_max - I)

In [79]:
test_I = np.linspace(4,4.99)
plt.figure()
plt.plot(test_I, [nu(I) for I in tqdm(test_I)])
plt.plot(test_I, [advanced_nu(I) for I in tqdm(test_I)])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [80]:
def current_point_advanced(t, I):
    if t == 0:
        return 0.0
    return -x(I) / (t * np.sqrt(2*np.pi*t)) * np.exp(-(x(I)+((advanced_nu(I)/2)*t))**2/(2*t))

In [81]:
def current_general_advanced(t, I, I_int_min):
    return [scipy.integrate.quad(lambda x: current_point_advanced(a_t, x), I_int_min, I)[0] for a_t in tqdm(t)]

In [75]:
plt.figure()
plt.plot(times, current)
plt.plot(times, current_general(times, I_0, 3.0))
plt.plot(times, current_general_advanced(times, I_0, 3.0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

  0%|          | 0/200 [00:00<?, ?it/s]